In [1]:
import h5py
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

In [2]:
class MRISliceGenerator(tf.keras.utils.Sequence):
    def __init__(self, file_list, batch_size=4, shuffle=True, use_dc=False):
        self.file_list = file_list
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.use_dc = use_dc   # NEW FLAG
        self.slice_index_map = []
        self._build_index()

    def _build_index(self):
        for file_idx, file_path in enumerate(self.file_list):
            with h5py.File(file_path, 'r') as f:
                num_slices = f['image_under'].shape[0]
                for slice_idx in range(num_slices):
                    self.slice_index_map.append((file_idx, slice_idx))
        self.on_epoch_end()

    def __len__(self):
        return int(np.ceil(len(self.slice_index_map) / self.batch_size))

    def __getitem__(self, index):
        batch_map = self.slice_index_map[index * self.batch_size:(index + 1) * self.batch_size]

        input_img_batch = []
        target_img_batch = []
        input_kspace_batch = []

        for file_idx, slice_idx in batch_map:
            with h5py.File(self.file_list[file_idx], 'r') as f:
                input_img = f['image_under'][slice_idx]       # (H,W,2)
                target_img = f['image_full'][slice_idx]       # (H,W,2)
                input_kspace = f['kspace_under'][slice_idx]   # (H,W,2)

                input_img_batch.append(input_img)
                target_img_batch.append(target_img)
                input_kspace_batch.append(input_kspace)

        x_img = np.stack(input_img_batch, axis=0)
        x_kspace = np.stack(input_kspace_batch, axis=0)
        y_batch = np.stack(target_img_batch, axis=0)

        if self.use_dc:
            # DSMENet expects two inputs when DC is used
            return [x_img, x_kspace], y_batch
        else:
            # Only image input (ZF)
            return x_img, y_batch

    def on_epoch_end(self):
        if self.shuffle:
            np.random.shuffle(self.slice_index_map)


In [3]:
val_folder = r"D:\fastmri_singlecoil_FSSCAN\val_norm"
import h5py
import numpy as np
import glob
import os

kspace_files_list_val = sorted(glob.glob(os.path.join(val_folder, "*.h5")))

# half_train = 20
# half_val = 10

half_val = len(kspace_files_list_val) 
# print("half_train",half_train)
# print("half_val",half_val)

kspace_files_list_val = kspace_files_list_val[:half_val]

# Create generators
# train_gen = MRISliceGenerator(kspace_files_list_train,batch_size=16, shuffle=True,mask=mask)
# val_gen = MRISliceGenerator(kspace_files_list_val, batch_size=4, shuffle=False,mask=mask)

val_gen = MRISliceGenerator(kspace_files_list_val, batch_size=4, shuffle=False)


print(len(val_gen))  


1784


In [4]:
%run model.ipynb

Model: "DSMENet_Functional"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_image (InputLayer)       [(None, 256, 256, 2  0           []                               
                                )]                                                                
                                                                                                  
 SRUN_1 (SRUN)                  ((None, 256, 256, 1  542204      ['input_image[0][0]']            
                                6),                                                               
                                 (None, 256, 256, 2                                               
                                ))                                                                
                                                                                 

In [5]:
model = build_DSMENet_functional(
    N=6, M=1, T=2,
    H=320, W=320, C=2
)

import numpy as np

In [8]:
import os
import time
import glob
import psutil
import numpy as np
import h5py
from tqdm import tqdm
import tensorflow as tf
from keras_flops import get_flops

# ============================================================
# CONFIGURATION
# ============================================================
VAL_FOLDER = r"D:\fastmri_singlecoil_FSSCAN\val_norm"
WARMUP_SLICES = 10
NUM_TIMING_SLICES = 100   # fixed slice count

# ============================================================
# FILE LIST
# ============================================================
file_paths = sorted(glob.glob(os.path.join(VAL_FOLDER, "*.h5")))
def inference_slice(model, x):
    _, F_final = model(x, training=False)
    #print("F_final",F_final.shape)
    return F_final


# ============================================================
# MODEL + INFERENCE FUNCTION MUST EXIST
# ============================================================
# model = ...
# def inference_slice(model, x): ...
assert model is not None, "Model is not loaded."
assert callable(inference_slice), "inference_slice() is not defined."

# ============================================================
# PARAMETER COUNT
# ============================================================
num_params = model.count_params()

# ============================================================
# FLOPs (PER SLICE, BATCH SIZE = 1)
# NOTE: FLOPs are counted for the model forward pass
# ============================================================
flops = get_flops(model, batch_size=1)

# ============================================================
# MEMORY HELPERS
# ============================================================
process = psutil.Process(os.getpid())
def cpu_memory_mb():
    return process.memory_info().rss / (1024 ** 2)

def gpu_memory_mb():
    info = tf.config.experimental.get_memory_info("GPU:0")
    return info["peak"] / (1024 ** 2)

# ============================================================
# LATENCY / THROUGHPUT MEASUREMENT
# ============================================================
def measure_latency(device):

    latencies = []

    with tf.device(device):

        # -----------------------------
        # WARM-UP
        # -----------------------------
        for file in file_paths[:1]:
            with h5py.File(file, "r") as f:
                image_under = f["image_under"][:]

            for s in range(min(WARMUP_SLICES, image_under.shape[0])):
                x = tf.convert_to_tensor(
                    image_under[s:s+1], dtype=tf.float32
                )
                _ = inference_slice(model, x)

        # -----------------------------
        # TIMED INFERENCE
        # -----------------------------
        count = 0
        for file in tqdm(file_paths, desc=f"Timing on {device}"):

            with h5py.File(file, "r") as f:
                image_under = f["image_under"][:]

            for s in range(image_under.shape[0]):

                if count >= NUM_TIMING_SLICES:
                    break

                x = tf.convert_to_tensor(
                    image_under[s:s+1], dtype=tf.float32
                )
                assert x.shape[0] == 1  # batch size = 1

                start = time.perf_counter()
                _ = inference_slice(model, x)

                # 🔑 GPU synchronization
                if "GPU" in device:
                    tf.config.experimental.get_memory_info("GPU:0")

                end = time.perf_counter()

                latencies.append(end - start)
                count += 1

            if count >= NUM_TIMING_SLICES:
                break

    latencies = np.array(latencies)

    mean_s = latencies.mean()
    median_s = np.median(latencies)

    return {
        "mean_s": mean_s,
        "median_s": median_s,
        "std_s": latencies.std(),
        "slices_per_sec": 1.0 / mean_s
    }

# ============================================================
# CPU BENCHMARK
# ============================================================
cpu_mem_before = cpu_memory_mb()
cpu_latency = measure_latency("/CPU:0")
cpu_mem_after = cpu_memory_mb()
cpu_mem_peak = cpu_mem_after - cpu_mem_before

# ============================================================
# GPU BENCHMARK (IF AVAILABLE)
# ============================================================
gpu_latency = None
gpu_mem_peak = None
gpu_name = None

gpus = tf.config.list_physical_devices("GPU")
if gpus:
    tf.config.experimental.reset_memory_stats("GPU:0")
    gpu_name = tf.config.experimental.get_device_details(gpus[0])["device_name"]
    gpu_latency = measure_latency("/GPU:0")
    gpu_mem_peak = gpu_memory_mb()

# ============================================================
# FINAL REPORT
# ============================================================
print("\n" + "=" * 70)
print("MODEL EFFICIENCY REPORT (SLICE-WISE, BATCH SIZE = 1)")
print("=" * 70)

print(f"Parameters: {num_params / 1e6:.2f} M")
print(f"FLOPs:      {flops / 1e9:.2f} GFLOPs (per slice)")

print("\n--- CPU Inference ---")
print(f"Latency:    {cpu_latency['mean_s']:.2f} s / slice")
print(f"Throughput:{cpu_latency['slices_per_sec']:.3f} slices/sec")
print(f"Memory:     {cpu_mem_peak:.2f} MB")

if gpu_latency:
    print("\n--- GPU Inference ---")
    print(f"GPU:        {gpu_name}")
    print(f"Latency:    {gpu_latency['mean_s']:.2f} s / slice")
    print(f"Throughput:{gpu_latency['slices_per_sec']:.3f} slices/sec")
    print(f"Peak VRAM:  {gpu_mem_peak:.2f} MB")
else:
    print("\nGPU not available.")

print("=" * 70)


Instructions for updating:
Use `tf.compat.v1.graph_util.tensor_shape_from_node_def_name`


Timing on /GPU:0:   1%|▌                                                             | 2/199 [02:03<3:23:15, 61.91s/it]


MODEL EFFICIENCY REPORT (SLICE-WISE, BATCH SIZE = 1)
Parameters: 1.21 M
FLOPs:      258.25 GFLOPs (per slice)

--- CPU Inference ---
Latency:    4.79 s / slice
Throughput:0.209 slices/sec
Memory:     5.51 MB

--- GPU Inference ---
GPU:        NVIDIA RTX A5000
Latency:    1.24 s / slice
Throughput:0.809 slices/sec
Peak VRAM:  269.08 MB


In [ ]:
def count_parameters_millions(model):
    trainable = np.sum([np.prod(v.shape) for v in model.trainable_variables])
    non_trainable = np.sum([np.prod(v.shape) for v in model.non_trainable_variables])
    total = trainable + non_trainable
    return (
        total / 1e6,
        trainable / 1e6,
        non_trainable / 1e6
    )

total_M, trainable_M, non_trainable_M = count_parameters_millions(model)

print("\n" + "=" * 40)
print(f"Total parameters:       {total_M:.3f} M")
print(f"Trainable parameters:   {trainable_M:.3f} M")
print(f"Non-trainable params:   {non_trainable_M:.3f} M")
print("=" * 40)
import tensorflow as tf
from tensorflow.python.framework.convert_to_constants import convert_variables_to_constants_v2

def compute_flops(model, input_shape):
    """
    input_shape: tuple, e.g. (1, H, W, 2)
    returns FLOPs (float operations) for one forward pass
    """

    @tf.function
    def forward(x):
        return model(x)

    concrete_func = forward.get_concrete_function(
        tf.TensorSpec(input_shape, tf.float32)
    )

    frozen_func = convert_variables_to_constants_v2(concrete_func)
    graph_def = frozen_func.graph.as_graph_def()

    with tf.Graph().as_default() as graph:
        tf.graph_util.import_graph_def(graph_def, name="")

        run_meta = tf.compat.v1.RunMetadata()
        opts = tf.compat.v1.profiler.ProfileOptionBuilder.float_operation()

        flops = tf.compat.v1.profiler.profile(
            graph=graph,
            run_meta=run_meta,
            cmd="op",
            options=opts
        )

    return flops.total_float_ops
# Example: infer H, W from your data or define explicitly
H, W = 320,320

flops = compute_flops(model, input_shape=(1, H, W, 2))

print(f"FLOPs (single forward pass): {flops / 1e9:.2f} GFLOPs")


Total parameters:       1.208 M
Trainable parameters:   1.208 M
Non-trainable params:   0.000 M
Instructions for updating:
Use `tf.compat.v1.graph_util.tensor_shape_from_node_def_name`
FLOPs (single forward pass): 258.04 GFLOPs


In [6]:
import numpy as np
import tensorflow as tf
from tqdm import tqdm

# =========================================================
# METRICS (SAME AS TRAINING)
# =========================================================

def nmse(pred, target):
    return tf.reduce_sum(tf.square(pred - target)) / tf.reduce_sum(tf.square(target))


# =========================================================
# CHECKPOINT LOADER
# =========================================================

def load_dmse_checkpoint(model, ckpt_dir="./checkpoints_dmse_full"):
    optimizer = tf.keras.optimizers.Adam()  # dummy optimizer

    epoch_counter = tf.Variable(0, dtype=tf.int64)
    best_val_ssim = tf.Variable(-1.0, dtype=tf.float32)

    ckpt = tf.train.Checkpoint(
        model=model,
        optimizer=optimizer,
        epoch=epoch_counter,
        best_val_ssim=best_val_ssim
    )

    manager = tf.train.CheckpointManager(
        ckpt,
        directory=ckpt_dir,
        max_to_keep=1
    )

    if not manager.latest_checkpoint:
        raise RuntimeError("❌ No checkpoint found!")

    ckpt.restore(manager.latest_checkpoint).expect_partial()

    print(f"\n✅ Restored checkpoint: {manager.latest_checkpoint}")
    print(f"⭐ Best Val SSIM: {best_val_ssim.numpy():.4f}")

    return model


# =========================================================
# INFERENCE STEP
# =========================================================

@tf.function
def inference_step(model, x):
    _, F_final = model(x, training=False)
    return F_final


# =========================================================
# EVALUATION LOOP (NO SAVING)
# =========================================================

def evaluate_dmse(model, val_gen):
    ssim_list = []
    psnr_list = []
    nmse_list = []

    val_bar = tqdm(range(len(val_gen)), desc="Evaluating", ncols=120)

    for step in val_bar:
        x_val, y_val = val_gen[step]

        # Forward pass
        F_final = inference_step(model, x_val)

        # Metrics
        ssim_val = tf.reduce_mean(tf.image.ssim(F_final, y_val, max_val=1.0))
        psnr_val = tf.reduce_mean(tf.image.psnr(F_final, y_val, max_val=1.0))
        nmse_val = nmse(F_final, y_val)

        ssim_list.append(ssim_val.numpy())
        psnr_list.append(psnr_val.numpy())
        nmse_list.append(nmse_val.numpy())

        val_bar.set_postfix({
            "SSIM": f"{ssim_val.numpy():.4f}",
            "PSNR": f"{psnr_val.numpy():.2f}",
            "NMSE": f"{nmse_val.numpy():.4f}"
        })

    # =====================================================
    # FINAL METRICS
    # =====================================================
    mean_ssim = float(np.mean(ssim_list))
    mean_psnr = float(np.mean(psnr_list))
    mean_nmse = float(np.mean(nmse_list))

    print("\n📊 FINAL VALIDATION RESULTS")
    print(f"  Mean SSIM : {mean_ssim:.4f}")
    print(f"  Mean PSNR : {mean_psnr:.2f}")
    print(f"  Mean NMSE : {mean_nmse:.4f}")

    return mean_ssim, mean_psnr, mean_nmse


# =========================================================
# RUN EVALUATION
# =========================================================


In [7]:

model = load_dmse_checkpoint(model)

mean_ssim, mean_psnr, mean_nmse = evaluate_dmse(model, val_gen)



✅ Restored checkpoint: ./checkpoints_dmse_full\ckpt-27
⭐ Best Val SSIM: 0.7191


Evaluating: 100%|█████████████████████████████| 1784/1784 [19:54<00:00,  1.49it/s, SSIM=0.6350, PSNR=29.57, NMSE=0.0579]


📊 FINAL VALIDATION RESULTS
  Mean SSIM : 0.7192
  Mean PSNR : 32.82
  Mean NMSE : 0.0829


In [10]:
val_folder = r"D:\fastmri_singlecoil_FSSCAN\val_norm"
kspace_files_list_val = sorted(glob.glob(os.path.join(val_folder, "*.h5")))
pd_files = []
pdfs_files = []

for f in kspace_files_list_val:
    if "PDFS" in f:
        pdfs_files.append(f)
    else:
        pd_files.append(f)

print(f"PD volumes: {len(pd_files)}")
print(f"PDFS volumes: {len(pdfs_files)}")

import os
import numpy as np
import h5py
import glob
from tqdm import tqdm
from skimage.metrics import peak_signal_noise_ratio, structural_similarity
def to_complex(x):
    return x[..., 0] + 1j * x[..., 1]

def nmse(gt, pred):
    return np.linalg.norm(gt - pred) ** 2 / (np.linalg.norm(gt) ** 2)

def compute_ssim(gt, pred, max_val):
    return structural_similarity(
        gt, pred,
        data_range=max_val,
        win_size=9,
        gaussian_weights=False,
        use_sample_covariance=False,
        K1=0.01,
        K2=0.03,
        full=False
    )
@tf.function
def inference_slice(model, x):
    _, F_final = model(x, training=False)
    #print("F_final",F_final.shape)
    return F_final

# ----------------------------------
# Containers for Excel
# ----------------------------------
volume_metrics = []   # volume-wise PSNR & NMSE
slice_metrics  = []   # slice-wise SSIM

psnr_list = []
nmse_list = []
ssim_list = []

for file in tqdm(pdfs_files, desc="Processing volumes", ncols=120):

    volume_name = os.path.basename(file)

    with h5py.File(file, 'r') as f:
        image_full  = f["image_full"][:]      # (S, H, W, 2)
        image_under = f["image_under"][:]     # (S, H, W, 2)
        max_val     = float(f["max_val_full_image"][0])

    # ----------------------
    # Slice-wise inference
    # ----------------------
    preds = []
    for s in range(image_under.shape[0]):
        x = tf.convert_to_tensor(image_under[s:s+1], dtype=tf.float32)
        pred = inference_slice(model, x)
        preds.append(pred.numpy()[0])

    pred = np.stack(preds, axis=0)  # (S, H, W, 2)

    # ----------------------
    # De-normalize
    # ----------------------
    image_full = image_full * max_val
    pred       = pred * max_val

    # ----------------------
    # Magnitude images
    # ----------------------
    gt_mag   = np.abs(to_complex(image_full))
    pred_mag = np.abs(to_complex(pred))

    # ----------------------
    # Volume-wise metrics
    # ----------------------
    nmse_val = nmse(gt_mag.flatten(), pred_mag.flatten())
    psnr_val = peak_signal_noise_ratio(gt_mag, pred_mag, data_range=max_val)

    nmse_list.append(nmse_val)
    psnr_list.append(psnr_val)

    volume_metrics.append({
        "Volume": volume_name,
        "PSNR (dB)": psnr_val,
        "NMSE": nmse_val
    })

    # ----------------------
    # Slice-wise SSIM
    # ----------------------
    for i in range(gt_mag.shape[0]):
        ssim_val = compute_ssim(gt_mag[i], pred_mag[i], max_val)

        ssim_list.append(ssim_val)

        slice_metrics.append({
            "Volume": volume_name,
            "Slice_ID": i,
            "SSIM": ssim_val
        })

# ----------------------------------
# Print summary
# ----------------------------------
print("\n" + "=" * 40)
print(f"PSNR (Mag, volume): {np.mean(psnr_list):.4f} ± {np.std(psnr_list):.4f} dB")
print(f"NMSE (Mag, volume): {np.mean(nmse_list):.6f} ± {np.std(nmse_list):.6f}")
print(f"SSIM (Mag, slice):  {np.mean(ssim_list):.4f} ± {np.std(ssim_list):.4f}")
print("=" * 40)

# ----------------------------------
# Save to Excel (multi-sheet)
# ----------------------------------


PD volumes: 100
PDFS volumes: 99


Processing volumes: 100%|███████████████████████████████████████████████████████████████| 99/99 [13:41<00:00,  8.30s/it]


PSNR (Mag, volume): -13.9292 ± 3.2128 dB
NMSE (Mag, volume): 1530.412936 ± 1501.141297
SSIM (Mag, slice):  0.0000 ± 0.0000


In [12]:
pd_files = []
pdfs_files = []

for f in kspace_files_list_val:
    if "PDFS" in f:
        pdfs_files.append(f)
    else:
        pd_files.append(f)

In [15]:
import os
import numpy as np
import h5py
import glob
from tqdm import tqdm
from skimage.metrics import peak_signal_noise_ratio, structural_similarity
def to_complex(x):
    return x[..., 0] + 1j * x[..., 1]

def nmse(gt, pred):
    return np.linalg.norm(gt - pred) ** 2 / (np.linalg.norm(gt) ** 2)

def compute_ssim(gt, pred, max_val):
    return structural_similarity(
        gt, pred,
        data_range=max_val,
        win_size=9,
        gaussian_weights=False,
        use_sample_covariance=False,
        K1=0.01,
        K2=0.03,
        full=False
    )

# ----------------------------------
# Containers for Excel
# ----------------------------------
volume_metrics = []   # volume-wise PSNR & NMSE
slice_metrics  = []   # slice-wise SSIM

psnr_list = []
nmse_list = []
ssim_list = []

for file in tqdm(pdfs_files, desc="Processing volumes", ncols=120):

    volume_name = os.path.basename(file)

    with h5py.File(file, 'r') as f:
        image_full  = f["image_full"][:]      # (S, H, W, 2)
        image_under = f["image_under"][:]     # (S, H, W, 2)
        max_val     = float(f["max_val_full_image"][0])

    # ----------------------
    # Slice-wise inference
    # ----------------------
    preds = []
    for s in range(image_under.shape[0]):
        x = tf.convert_to_tensor(image_under[s:s+1], dtype=tf.float32)
        pred = inference_slice(model, x)
        preds.append(pred.numpy()[0])

    pred = np.stack(preds, axis=0)  # (S, H, W, 2)

    # ----------------------
    # De-normalize
    # ----------------------
    image_full = image_full * max_val
    pred       = pred * max_val

    # ----------------------
    # Magnitude images
    # ----------------------
    gt_mag   = np.abs(to_complex(image_full))
    pred_mag = np.abs(to_complex(pred))

    # ----------------------
    # Volume-wise metrics
    # ----------------------
    nmse_val = nmse(gt_mag.flatten(), pred_mag.flatten())
    psnr_val = peak_signal_noise_ratio(gt_mag, pred_mag, data_range=max_val)

    nmse_list.append(nmse_val)
    psnr_list.append(psnr_val)

    volume_metrics.append({
        "Volume": volume_name,
        "PSNR (dB)": psnr_val,
        "NMSE": nmse_val
    })

    # ----------------------
    # Slice-wise SSIM
    # ----------------------
    for i in range(gt_mag.shape[0]):
        ssim_val = compute_ssim(gt_mag[i], pred_mag[i], max_val)

        ssim_list.append(ssim_val)

        slice_metrics.append({
            "Volume": volume_name,
            "Slice_ID": i,
            "SSIM": ssim_val
        })

# ----------------------------------
# Print summary
# ----------------------------------
print("\n" + "=" * 40)
print(f"PSNR (Mag, volume): {np.mean(psnr_list):.4f} ± {np.std(psnr_list):.4f} dB")
print(f"NMSE (Mag, volume): {np.mean(nmse_list):.6f} ± {np.std(nmse_list):.6f}")
print(f"SSIM (Mag, slice):  {np.mean(ssim_list):.4f} ± {np.std(ssim_list):.4f}")
print("=" * 40)

# ----------------------------------
# Save to Excel (multi-sheet)
# ----------------------------------


Processing volumes: 100%|███████████████████████████████████████████████████████████████| 99/99 [13:21<00:00,  8.10s/it]


PSNR (Mag, volume): 32.5894 ± 2.5905 dB
NMSE (Mag, volume): 0.029795 ± 0.013134
SSIM (Mag, slice):  0.7649 ± 0.0809


In [12]:
def load_volume(h5_path):
    with h5py.File(h5_path, "r") as f:
        x = f["image_under"][:]     # [S, H, W, C]
        y = f["image_full"][:]   # [S, H, W, C]
    return x, y


In [9]:
@tf.function
def inference_slice(model, x):
    _, F_final = model(x, training=False)
    #print("F_final",F_final.shape)
    return F_final


Processing volumes: 100%|█████████████████████████████████████████████████████████████| 199/199 [24:32<00:00,  7.40s/it]


PSNR (Mag, volume): 33.3280 ± 2.7036 dB
NMSE (Mag, volume): 0.022481 ± 0.012902
SSIM (Mag, slice):  0.7894 ± 0.0771


NameError: name 'pd' is not defined

In [16]:
import pandas as pd
df_volume = pd.DataFrame(volume_metrics)
df_slice  = pd.DataFrame(slice_metrics)

excel_path = "validation_metrics_DMSENet.xlsx"

with pd.ExcelWriter(excel_path, engine="openpyxl") as writer:
    df_volume.to_excel(writer, sheet_name="Volume_PSNR_NMSE", index=False)
    df_slice.to_excel(writer, sheet_name="Slice_SSIM", index=False)

print(f"\nSaved metrics to: {excel_path}")



Saved metrics to: validation_metrics_DMSENet.xlsx


In [27]:
def to_complex(x):
    return x[..., 0] + 1j * x[..., 1]

def nmse(gt, pred):
    return np.linalg.norm(gt - pred) ** 2 / (np.linalg.norm(gt) ** 2)

def compute_ssim(gt, pred, max_val):
    return structural_similarity(
        gt, pred,
        data_range=max_val,
        win_size=9,
        gaussian_weights=False,
        use_sample_covariance=False,
        K1=0.01,
        K2=0.03,
        full=False
    )
ssim_list = []
psnr_list = []
nmse_list = []
for file in tqdm(kspace_files_list_val, desc="Processing volumes", ncols=120):

    with h5py.File(file, 'r') as f:
        image_full  = f["image_full"][:]      # (S, H, W, 2)
        image_under = f["image_under"][:]     # (S, H, W, 2)
        max_val     = float(f["max_val_full_image"][0])

    # ----------------------
    # Slice-wise inference
    # ----------------------
    preds = []

    for s in range(image_under.shape[0]):
        x = tf.convert_to_tensor(image_under[s:s+1], dtype=tf.float32)
        pred = inference_slice(model, x)
        preds.append(pred.numpy()[0])

    pred = np.stack(preds, axis=0)  # (S, H, W, 2)

    # ----------------------
    # De-normalize
    # ----------------------
    image_full = image_full * max_val
    pred = pred * max_val

    # ----------------------
    # Magnitude images
    # ----------------------
    gt_mag   = np.abs(to_complex(image_full))
    pred_mag = np.abs(to_complex(pred))

    # ----------------------
    # Volume-wise NMSE & PSNR
    # ----------------------
    nmse_val = nmse(gt_mag.flatten(), pred_mag.flatten())
    psnr_val = peak_signal_noise_ratio(gt_mag, pred_mag, data_range=max_val)
    #print("psnr_val",psnr_val)
    #print("nmse_val",nmse_val)
    nmse_list.append(nmse_val)
    psnr_list.append(psnr_val)

    # ----------------------
    # Slice-wise SSIM
    # ----------------------
    for i in range(gt_mag.shape[0]):
        ssim_val = compute_ssim(gt_mag[i], pred_mag[i], max_val)
        ssim_list.append(ssim_val)
        #print("ssim_val",ssim_val)
print("\n" + "=" * 40)
print(f"PSNR (Mag, volume): {np.mean(psnr_list):.4f} ± {np.std(psnr_list):.4f} dB")
print(f"NMSE (Mag, volume): {np.mean(nmse_list):.6f} ± {np.std(nmse_list):.6f}")
print(f"SSIM (Mag, slice):  {np.mean(ssim_list):.4f} ± {np.std(ssim_list):.4f}")
print("=" * 40)


Processing volumes: 100%|█████████████████████████████████████████████████████████████| 199/199 [30:15<00:00,  9.12s/it]


PSNR (Mag, volume): 32.6027 ± 3.7079 dB
NMSE (Mag, volume): 0.064605 ± 0.054789
SSIM (Mag, slice):  0.7834 ± 0.0784


In [16]:
from tqdm import tqdm

def evaluate_dmse_volume_wise(model, h5_files):
    vol_ssim_all = []
    vol_psnr_all = []
    vol_nmse_all = []

    vol_bar = tqdm(h5_files, desc="Evaluating volumes", ncols=120)

    for h5_path in vol_bar:
        x_vol, y_vol = load_volume(h5_path)

        slice_ssim = []
        slice_psnr = []
        slice_nmse = []

        for s in range(x_vol.shape[0]):
            x_slice = tf.convert_to_tensor(x_vol[s:s+1])  # [1,H,W,C]
            y_slice = tf.convert_to_tensor(y_vol[s:s+1])

            F_final = inference_slice(model, x_slice)

            ssim = tf.image.ssim(F_final, y_slice, max_val=1.0)
            psnr = tf.image.psnr(F_final, y_slice, max_val=1.0)
            nmse_val = nmse(F_final, y_slice)

            slice_ssim.append(ssim.numpy()[0])
            slice_psnr.append(psnr.numpy()[0])
            slice_nmse.append(nmse_val.numpy())

        # Per-volume mean
        vol_ssim = np.mean(slice_ssim)
        vol_psnr = np.mean(slice_psnr)
        vol_nmse = np.mean(slice_nmse)

        vol_ssim_all.append(vol_ssim)
        vol_psnr_all.append(vol_psnr)
        vol_nmse_all.append(vol_nmse)

        vol_bar.set_postfix({
            "SSIM": f"{vol_ssim:.4f}",
            "PSNR": f"{vol_psnr:.2f}",
            "NMSE": f"{vol_nmse:.4f}"
        })

    print("\n📊 FINAL VOLUME-WISE METRICS")
    print(f"Mean SSIM : {np.mean(vol_ssim_all):.4f}")
    print(f"Mean PSNR : {np.mean(vol_psnr_all):.2f}")
    print(f"Mean NMSE : {np.mean(vol_nmse_all):.4f}")

    return (
        float(np.mean(vol_ssim_all)),
        float(np.mean(vol_psnr_all)),
        float(np.mean(vol_nmse_all))
    )


In [ ]:
model = load_dmse_checkpoint(model)

mean_ssim, mean_psnr, mean_nmse = evaluate_dmse_volume_wise(
    model,
    kspace_files_list_val
)
